In [ ]:
%matplotlib inline

In [ ]:
"""
Dedalus script for 2D tracer advection-diffusion on a slope

This script uses a Fourier basis in the y direction with periodic boundary
conditions.

This script can be ran serially or in parallel, and uses the built-in analysis
framework to save data snapshots in HDF5 files.  The `merge.py` script in this
folder can be used to merge distributed analysis sets from parallel runs,
and the `plot_2d_series.py` script can be used to plot the snapshots.

To run, merge, and plot using 4 processes, for instance, you could use:
    $ mpiexec -n 4 python3 igw.py
    $ mpiexec -n 4 python3 merge.py snapshots
    $ mpiexec -n 4 python3 plot_2d_series.py snapshots/*.h5

"""
import numpy as np
import h5py
import matplotlib
import shutil
# from matplotlib import rc
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
# rc('text', usetex=True)
matplotlib.rcParams["figure.facecolor"] = "white"
matplotlib.rcParams["axes.facecolor"] = "white"
matplotlib.rcParams["savefig.facecolor"] = "white"
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpi4py import MPI
from scipy.special import erf
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

from dedalus.tools import post
import pathlib
from dedalus.extras import plot_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:

# Input Grids and parameters ------------------------------------------------------------
Ly, Lz = (400000., 2000.) # units = 1m

# Create bases and domain
y_basis = de.Fourier('y', 256, interval=(0, Ly), dealias=3/2)
z_basis = de.Chebyshev('z', 128, interval=(0, Lz), dealias=3/2)
domain = de.Domain([y_basis, z_basis], grid_dtype=np.float64)

# Input fields --------------------------------------------------------------------------
y = domain.grid(0)
z = domain.grid(1)

N2 = 1.0e-6
slope = 1/400
theta = np.arctan(slope)

# Vertical Diffusivity
Kv = domain.new_field()
Kv.meta['y']['constant'] = True
Kvz = domain.new_field()
Kvz.meta['y']['constant'] = True
Kvref = domain.new_field()
Kvref.meta['y']['constant'] = True
Kvzref = domain.new_field()
Kvzref.meta['y']['constant'] = True

K0 = 5.0e-4
d = 500.0
K1 = 2.0e-3
Kv['g'] = K1*np.exp(-z/d)
Kv.differentiate('z',out=Kvz)

n = 7.
Kvref['g'] = K1*(1-z/d/n)**n
Kvref.differentiate('z',out=Kvzref)

# Horizontal Diffusivity
Ev = domain.new_field()
Ev.meta['y']['constant'] = True

E0 = 0.0
Ev['g'] = 0.0

# Upslope Velocity
PSI = domain.new_field()
PSI.meta['y']['constant'] = True
V = domain.new_field()
V.meta['y']['constant'] = True

q = 1.0/100.0
PSI['g'] = K0*np.cos(theta)/np.sin(theta)*(1-np.exp(-q*z)*(np.cos(q*z)+np.sin(q*z)))
#PSI['g'] = 0.0
#Bz['g'] = PSI['g']*N2*np.sin(theta)/(Kv['g']+K0)
PSI.differentiate('z',out=V)

# Buoyancy for analysis
Bz = domain.new_field()
Bz.meta['y']['constant'] = True
B = domain.new_field()
B['g'] = N2*np.sin(theta)*y + N2*np.cos(theta)*(z + np.exp(-q*z)*np.cos(q*z)/q)
B.differentiate('z',out=Bz)
#Bz.integrate('z',out=B)
#B['g'] = B['g'] + N2*np.sin(theta)*y

# Equations and Solver
problem = de.IVP(domain, variables=['tr','trz'])
problem.meta[:]['z']['dirichlet'] = True

problem.parameters['E0'] = E0
problem.parameters['K0'] = K0
problem.parameters['Ev'] = Ev
problem.parameters['Kv'] = Kv
problem.parameters['Kvz'] = Kvz
problem.parameters['Kvref'] = Kvref
problem.parameters['Kvzref'] = Kvzref
problem.parameters['V'] = V
problem.parameters['B'] = B
#problem.add_equation("dt(tr) - E0*d(tr,y=2) - K0*dz(trz) = Kv*dz(trz) + Kvz*trz + Ev*d(tr,y=2) - V*dy(tr)")
problem.add_equation("dt(tr) - E0*d(tr,y=2) - K0*dz(trz) -Kvref*dz(trz) - Kvzref*trz = (Kv-Kvref)*dz(trz) + (Kvz-Kvzref)*trz + Ev*d(tr,y=2) - V*dy(tr)")
problem.add_equation("trz - dz(tr) = 0")
problem.add_bc("left(trz) = 0")
problem.add_bc("right(trz) = 0")

# Build solver
solver = problem.build_solver(de.timesteppers.RK222)
logger.info('Solver built')

# Initial condition:
tr = solver.state['tr']
trz = solver.state['trz']

# Gaussian blob:
sy = Ly/20
sz = Lz/16
cy = Ly/3
cz = Lz/4
tr['g'] = np.exp(-(z-cz)**2/2/sz**2 -(y-cy)**2/2/sy**2)
#tr['g'] = np.exp(-(y-cy)**2/2/sy**2)
#tr['g'] = np.exp(-(z-cz)**2/2/sz**2)

# Function of buoyancy:
# tr['g'] = 0*z
# tr['g'][np.logical_and(B['g']/N2/np.cos(theta)>=Lz/4,B['g']/N2/np.cos(theta)<=Lz/4+200.)] = 1

tr.differentiate('z',out=trz)

# Integration parameters
lday = 1.0e5 # A "long-day" unit (86400 ~= 100000)
#dt=0.0025*lday#2*lday
dt=2*lday
# 2*lday works with constant diffusion
# 0.0025*lday works with exp diffusion (factor 800 smaller).
# 2*lday works with exp diffusion if using n=7 poly approx.
solver.stop_sim_time = np.inf
solver.stop_wall_time = np.inf 
solver.stop_iteration = 1600*lday/dt
#solver.stop_iteration = 800*lday/dt

# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', iter=10, max_writes=500)
snapshots.add_system(solver.state, layout='g')
snapshots.add_task("B", layout='g', name = 'B')
snapshots.add_task("integ(tr,'y')", layout='g', name = 'ym0')
snapshots.add_task("integ(tr*y,'y')", layout='g', name = 'ym1')
snapshots.add_task("integ(tr*y*y,'y')", layout='g', name = 'ym2')
snapshots.add_task("integ(tr,'z')", layout='g', name = 'zm0')
snapshots.add_task("integ(tr*z,'z')", layout='g', name = 'zm1')
snapshots.add_task("integ(tr*z*z,'z')", layout='g', name = 'zm2')
snapshots.add_task("integ(integ(tr,'y'),'z')", layout = 'g', name = 'trT')
snapshots.add_task("integ(integ(tr*B,'y'),'z')", layout = 'g', name = 'bm1i')
snapshots.add_task("integ(integ(tr*B*B,'y'),'z')", layout = 'g', name = 'bm2i')
snapshots.add_task("integ(integ(tr*y,'y'),'z')", layout='g', name = 'ym1i')
snapshots.add_task("integ(integ(tr*y*y,'y'),'z')", layout='g', name = 'ym2i')
snapshots.add_task("integ(integ(tr*z,'z'),'y')", layout='g', name = 'zm1i')
snapshots.add_task("integ(integ(tr*z*z,'z'),'y')", layout='g', name = 'zm2i')


In [ ]:
# Run the model and simple plot rotated tracer concentration as it runs:
y = domain.grid(0,scales=domain.dealias);z = domain.grid(1,scales=domain.dealias); # Native Coordinates
ym, zm = np.meshgrid(y,z)
zt = np.cos(theta)*zm + np.sin(theta)*ym;yt = -np.sin(theta)*zm + np.cos(theta)*ym # Rotated Coordinates

# Setup figure:
#fig = plt.figure(figsize=(15, 5), facecolor='w')
fig = plt.figure(figsize=(30, 10), facecolor='w')

axisV = plt.subplot2grid((1,6),(0,0), colspan=1)
axisV.plot(V['g'][0,:]*100.0,z.T, linewidth=2);axisV.set_xlabel('Upslope Velocity (cms-1)');axisV.set_ylabel('z (m)'); axisV.set_ylim([0.,Lz])

axisK = plt.subplot2grid((1,6),(0,1), colspan=1);axisK.plot((Kv['g'][0,:]+K0)/1.0e-4,z.T, linewidth=2);axisK.set_xlabel('z Diffusivity (cm2s-1)');axisK.set_ylim([0.,Lz])

axisTR = plt.subplot2grid((1,6),(0,2),colspan=4)
p = axisTR.pcolormesh(yt/1.0e3, zt, tr['g'].T, cmap='RdBu_r', vmin=0., vmax=0.2);
#p = axisTR.pcolormesh(ym/1.e3, zm, tr['g'].T/np.max(tr['g']), cmap='RdBu_r', vmin=0., vmax=1.);

# Add buoyancy contours
B = N2*np.sin(theta)*ym + N2*np.cos(theta)*(zm + np.exp(-q*zm)*np.cos(q*zm)/q)
plt.contour(yt/1.0e3, zt, B, 30, colors='k',linewidth=1)
#plt.contour(yt/1.0e3, zt, B['g'].T, 30, colors='k',linewidth=1)
axisTR.plot(y/1.0e3, slope*y,'k-', linewidth=4)
plt.colorbar(p, ax = axisTR)
axisTR.set_xlabel('True y (km)');axisTR.set_ylabel('True z (m)');axisTR.set_title('Tracer Concentration')
axisTR.set_xlim([0,Ly/1.e3]);axisTR.set_ylim([0.,Lz + slope*Ly]);axisTR.set_facecolor('k')
plt.savefig('CurrentRunIC')
    
# Main loop
try:
    logger.info('Starting loop')
    start_time = time.time()
    while solver.ok:
        #        dt = CFL.compute_dt()
        solver.step(dt)
        if (solver.iteration-1) % 10 == 0:
            logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))
        if (solver.iteration-1) % 100 == 0:
            p.set_array(np.ravel(tr['g'][:-1,:-1].T))
#            p.set_array(np.ravel(tr['g'][:-1,:-1].T/np.max(tr['g'])))
            display.clear_output()
            display.display(plt.gcf())
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    end_time = time.time()

    p.set_array(np.ravel(tr['g'][:-1,:-1].T))
#    p.set_array(np.ravel(tr['g'][:-1,:-1].T/np.max(tr['g'])))
    display.clear_output()
    logger.info('Iterations: %i' %solver.iteration)
    logger.info('Sim end time: %f' %solver.sim_time)
    logger.info('Run time: %.2f sec' %(end_time-start_time))
    logger.info('Run time: %f cpu-hr' %((end_time-start_time)/60/60*domain.dist.comm_cart.size))
    plt.savefig('CurrentRunLast')


In [ ]:
# POST-PROCESSING:

# Merge snapshots from different processes:
post.merge_process_files("snapshots", cleanup=True)
# Merge snapshots from different output sets:
set_paths = list(pathlib.Path("snapshots").glob("snapshots_s*.h5"))
post.merge_sets("snapshots/snapshots.h5", set_paths, cleanup=True)

In [ ]:
#outname = 'zDiffusion1em3_andBLV'
#outname = 'PurezDiffusion1em3'
outname = 'zDiffusion1em4_andBLV_andexpK1em3'


In [ ]:
# Plot B distribution and moments:
fig = plt.figure(figsize=(20, 10), facecolor='w')
ab0 = plt.subplot2grid((3,2),(0,0), colspan=1, rowspan=3)
ab1 = plt.subplot2grid((3,2),(0,1));
ab2 = plt.subplot2grid((3,2),(1,1));
ab3 = plt.subplot2grid((3,2),(2,1));
ab0.set_ylabel('b (10^{-4} ms^{-2})');ab0.set_xlabel('Tracer Concentration (tr / (ms^{-2}))');
ab0.set_title('Tracer Distribution')
ab0.set_ylim([0.,20.]);
ab1.set_ylabel('b (10^{-4} ms^{-2})');ab1.set_title('Centre of mass \mu_b');
ab2.set_ylabel('b (10^{-4} ms^{-2})');ab2.set_title('Standard deviation \sigma_b');
ab3.set_ylabel('\kappa (cm^2s^{-1})');ab3.set_title('Effective diffusivity from second moment');
eps = 0.1;

b = np.linspace(0,N2*np.cos(theta)*Lz + N2*np.sin(theta)*Ly,64);db = b[1]-b[0]
ba = (b[:-1]+b[1:])/2

with h5py.File("snapshots/snapshots.h5", mode='r') as file:

    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
    dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
    dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);
    t = file['scales/sim_time'][:]/lday;
    B = file['tasks']['B'][0,:,:]
    B = (B[1:,:] + B[:-1,:])/2
    B = (B[:,1:] + B[:,:-1])/2

    mub = np.zeros(len(t))
    sigma2b = np.zeros(len(t))
    for i in range(0,len(t),2):
        tr = file['tasks']['tr'][i,:,:]
        tr = (tr[1:,:] + tr[:-1,:])/2
        tr = (tr[:,1:] + tr[:,:-1])/2
        tr = tr*dA
        trC = [np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]]
#        mub[i] = np.sum(trC*ba)/np.sum(trC)
#        sigma2b[i] = np.sum(trC*ba*ba)/np.sum(trC) - mub[i]**2
        ab0.plot(trC,(b[:-1]+b[1:])/2/1.e-4)
    
        
    bm1 = file['tasks']['bm1i'][:,0,0];
    bm2 = file['tasks']['bm2i'][:,0,0]
#    ym1 = file['tasks']['ym1i'][:,0,0];
#    zm1 = file['tasks']['zm1i'][:,0,0]
    trT = file['tasks']['trT'][:,0,0]
    mu = bm1/trT
    sigma2 = bm2/trT-(bm1/trT)**2.
    ab1.plot(t,mu/1.e-4)
#    ab1.plot(t,mub/1.e-4)
#    ab1.plot(t,ym1/trT*N2*np.sin(theta)/1.e-4+zm1/trT*N2*np.cos(theta)/1.e-4-mu/1.e-4)
    ab2.plot(t,np.sqrt(sigma2)/1.e-4)
#    ab2.plot(t,np.sqrt(sigma2b)/1.e-4)

    ab3.plot((t[1:]+t[:-1])/2,0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/N2/N2/1.e-4)
#    ab3.plot((t[1:]+t[:-1])/2,0.5*(sigma2b[1:]-sigma2b[:-1])/(t[1:]-t[:-1])/lday/N2/N2/1.e-4)
plt.savefig(outname + '_bSpaceAnalysis');


In [ ]:
# Plot Z distribution and moments:
fig = plt.figure(figsize=(20, 10), facecolor='w')
ab0 = plt.subplot2grid((3,2),(0,0), colspan=1, rowspan=3)
ab1 = plt.subplot2grid((3,2),(0,1));
ab2 = plt.subplot2grid((3,2),(1,1));
ab3 = plt.subplot2grid((3,2),(2,1));
ab0.set_ylabel('z (m)');ab0.set_xlabel('Tracer Concentration (tr / m)');
ab0.set_title('Tracer Distribution');ab0.set_ylim([0.,Lz]);
ab1.set_ylabel('z (m)');ab1.set_title('Centre of mass \mu_z');
ab2.set_ylabel('z (m)');ab2.set_title('Standard deviation \sigma_z');
ab3.set_ylabel('\kappa (cm^2s^{-1})');ab3.set_title('Effective diffusivity from second moment');
eps = 0.1;

with h5py.File("snapshots/snapshots.h5", mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    ym0 = file['tasks']['ym0']

    for i in range(0,len(t),2):
        ab0.plot(ym0[i,0,:],z);
    
    zm1 = file['tasks']['zm1i'][:,0,0];
    zm2 = file['tasks']['zm2i'][:,0,0]
    trT = file['tasks']['trT'][:,0,0]
    mu = zm1/trT
    sigma2 = zm2/trT-(zm1/trT)**2.
    ab1.plot(t,mu)
    ab2.plot(t,np.sqrt(sigma2))

    ab3.plot((t[1:]+t[:-1])/2,0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday/1e-4)
plt.savefig(outname + '_zSpaceAnalysis');

In [ ]:
# Plot Y distribution and moments:
fig = plt.figure(figsize=(20, 10), facecolor='w')
ab0 = plt.subplot2grid((3,2),(0,0), colspan=1, rowspan=3)
ab1 = plt.subplot2grid((3,2),(0,1));
ab2 = plt.subplot2grid((3,2),(1,1));
ab3 = plt.subplot2grid((3,2),(2,1));
ab0.set_xlabel('y (km)');ab0.set_ylabel('Tracer Concentration (tr / m)');
ab0.set_title('Tracer Distribution');ab0.set_xlim([0.,Ly/1.e3]);
ab1.set_ylabel('y (km)');ab1.set_title('Centre of mass \mu_y');
ab2.set_ylabel('y (km)');ab2.set_title('Standard deviation \sigma_y');
ab3.set_ylabel('\kappa (m^2s^{-1})');ab3.set_title('Effective diffusivity from second moment');
eps = 0.1;

with h5py.File("snapshots/snapshots.h5", mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    zm0 = file['tasks']['zm0']

    for i in range(0,len(t),2):
        ab0.plot(y/1.e3,zm0[i,:,0]);
    
    ym1 = file['tasks']['ym1i'][:,0,0];
    ym2 = file['tasks']['ym2i'][:,0,0]
    trT = file['tasks']['trT'][:,0,0]
    mu = ym1/trT
    sigma2 = ym2/trT-(ym1/trT)**2.
    ab1.plot(t,mu/1.e3)
    ab2.plot(t,np.sqrt(sigma2)/1.e3)

    ab3.plot((t[1:]+t[:-1])/2,0.5*(sigma2[1:]-sigma2[:-1])/(t[1:]-t[:-1])/lday)
plt.savefig(outname + '_ySpaceAnalysis');

In [ ]:
# Plot Y and Z moments on top of each other:
fig = plt.figure(figsize=(25, 10), facecolor='w')
ay0 = plt.subplot2grid((3,6),(0,0), rowspan=3);ay1 = plt.subplot2grid((3,6),(0,1), rowspan=3);ay2 = plt.subplot2grid((3,6),(0,2), rowspan=3)
ay0.set_ylabel('z (m)');ay0.set_title('< tr >^y');
ay1.set_title('< y tr >^y / < tr >^y');
ay2.set_title('< y^2 tr >^y / < tr >^y - (< y tr >^y / < tr >^y)^2');
eps = 0.1;

az0 = plt.subplot2grid((3,6),(0,3), colspan=3);az1 = plt.subplot2grid((3,6),(1,3), colspan=3);az2 = plt.subplot2grid((3,6),(2,3), colspan=3)
az2.set_xlabel('y (m)');az0.set_title('< tr >^z');
az1.set_title('< z tr >^z / < tr >^z');
az2.set_title('< z^2 tr >^z / < tr >^z - (< z tr >^z / < tr >^z)^2');

with h5py.File("snapshots/snapshots.h5", mode='r') as file:
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:]
    ym0 = file['tasks']['ym0'];ym1 = file['tasks']['ym1'];ym2 = file['tasks']['ym2']
    zm0 = file['tasks']['zm0'];zm1 = file['tasks']['zm1'];zm2 = file['tasks']['zm2']

    for i in range(len(ym0[:,0,0])):
        ay0.plot(ym0[i,0,:],z)
        ay1.plot(ym1[i,0,:]/np.maximum(ym0[i,0,:],eps)/1.e3,z)
        ay2.plot(np.sqrt(ym2[i,0,:]/np.maximum(ym0[i,0,:],eps)-(ym1[i,0,:]/np.maximum(ym0[i,0,:],eps))**2.)/1.0e3,z)

        az0.plot(y/1.e3,zm0[i,:,0])
        az1.plot(y/1.e3,zm1[i,:,0]/np.maximum(zm0[i,:,0],eps))
        az2.plot(y/1.e3,np.sqrt(zm2[i,:,0]/np.maximum(zm0[i,:,0],eps)-(zm1[i,:,0]/np.maximum(zm0[i,:,0],eps))**2.))
plt.savefig('YZMoments')

In [ ]:
with h5py.File("snapshots/snapshots.h5", mode='r') as file:
    print(list(file['tasks']))